<a href="https://colab.research.google.com/github/cfoli/Multi-label-Medical-Image-Classification/blob/main/Multi_label_CNN_chestXray_classification_6label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import libraries

---



In [ ]:
!pip install lightning torchmetrics

In [2]:
import torch
import torch.nn.functional as F
# import torchvision.datasets as datasets
import torchvision
from torchvision import transforms, models, datasets
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split
import pytorch_lightning as torch_light
from pytorch_lightning.callbacks import EarlyStopping
import torchmetrics
from torchmetrics import Metric
import os
import shutil
import subprocess
import pandas as pd


### Define helper functions and containers

---



In [3]:
BASE_DIR = os.path.join(os.getcwd(), "datasets")

configs = {
    # Image & label properties
    "IMAGE_SIZE":   (224, 224),          # Resize images to (W, H)
    "NUM_CHANNELS": 3,                     # RGB images
    "NUM_CLASSES":  15,                 # Number of output labels
    # "VALID_PCT":    0.1,

    "BATCH_SIZE": 32,

    # ImageNet dataset normalization values (for pretrained backbones)
    "MEAN": (0.485, 0.456, 0.406),
    "STD":  (0.229, 0.224, 0.225),

    "LABELS_CSV_DIR": os.path.join(BASE_DIR, "new_labels.csv"),
    "IMG_DIR": os.path.join(BASE_DIR, "resized_images", "resized_images"),

    "TRAIN_PCT": 0.8, # Fraction of data for training
    "VAL_PCT":   0.1, # Fraction of data for validation

    # Dataset paths
    # "TRAIN_DATA_DIR": os.path.join(BASE_DIR, "chest-xray", "train"),
    # "TEST_DATA_DIR":  os.path.join(BASE_DIR, "chest-xray", "test"),
    # "VAL_DATA_DIR":   os.path.join(BASE_DIR, "chest-xray", "val")
    }

labels_dict = {
    0: "Atelectasis",
    1: "Cardiomegaly",
    2: "Consolidation",
    3: "Edema",
    4: "Effusion",
    5: "Emphysema",
    6: "Fibrosis",
    7: "Hernia",
    8: "Infiltration",
    9: "Mass",
    10: "No finding",
    11: "Nodule",
    11: "Pleural_Thickening",
    12: "Pneumonia",
    13: "Pneumothorax"}

def get_pretrained_model(pretrained_model_name: str, num_classes: int, freeze_backbone: bool = True):
    """
    Load a pretrained Torchvision classification model and replace
    ONLY its final Linear layer for transfer learning or fine-tuning.
    """

    print(f"Loading pretrained [{pretrained_model_name}] model")

    # Load pretrained model from torchvision
    model = getattr(torchvision.models, pretrained_model_name)(weights="DEFAULT")

    # Optionally freeze all pretrained parameters (the backbone)
    if freeze_backbone:
        for param in model.parameters():
            param.requires_grad = False

    # Get the last top-level module (typically the classifier head) from the pretrained model
    layer_modules = list(model.named_children())
    last_module_name, last_module = layer_modules[-1]

    if isinstance(last_module, nn.Sequential):
        # If the classifier is a Sequential module, replace its final layer

        num_in_features = last_module[-1].in_features # find the dimensionality of the input to the last layer of the network
        last_module[-1] = nn.Linear(num_in_features, num_classes)

    else:
        # Otherwise, replace the module directly (e.g., ResNet-style fc layer)
        in_features = last_module.in_features
        setattr(model, last_module_name, nn.Linear(in_features, num_classes))

    return model

### Load kaggle credentials (i.e., API) file

---



In [4]:
# Import kaggle API token file
from google.colab import files
files.upload() # This will prompt you to select and upload the file

# Then move and secure the file
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


### Create model (i.e. classifier) module

---



In [5]:
class modelModule(torch_light.LightningModule):
    def __init__(self, num_classes, backbone_model_name):
        super().__init__()
        self.num_classes = num_classes
        self.backbone_model_name = backbone_model_name

        # Load a pretrained backbone and replace its final layer
        self.model = get_pretrained_model(
            num_classes = self.num_classes,
            pretrained_model_name = self.backbone_model_name )

        # Binary classification loss operating on raw logits
        self.loss_function = torch.nn.BCEWithLogitsLoss()
        self.accuracy_function = torchmetrics.Accuracy(task="multilabel", num_labels=self.num_classes)
        self.f1_score_function = torchmetrics.F1Score(task="multilabel", num_labels=self.num_classes)

    def forward(self, x):
        # Forward pass through the backbone model
        return self.model(x)

    def _common_step(self, batch, batch_idx):
        """
        Shared logic for train / val / test steps.
        Computes loss and evaluation metrics.
        """
        x, y = batch

        # Compute model predictions ()
        y_hat = self.forward(x)

        # Compute metrics (expects logits + labels)
        loss     = self.loss_function(y_hat, y)
        accuracy = self.accuracy_function(y_hat, y)
        f1_score = self.f1_score_function(y_hat, y)

        return loss, y_hat, y, accuracy, f1_score

    def training_step(self, batch, batch_idx):
        # Run shared step
        loss, y_hat, y, accuracy, f1_score = self._common_step(batch, batch_idx)

        # Log epoch-level training metrics
        self.log_dict(
            {"train_loss": loss, "train_accuracy": accuracy, "train_f1_score": f1_score},
            on_step=False, on_epoch=True, prog_bar=True)

        # Lightning expects the loss key for backprop
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        # Run shared step
        loss, y_hat, y, accuracy, f1_score = self._common_step(batch, batch_idx)

        # Log validation metrics
        self.log_dict(
            {"val_loss": loss, "val_accuracy": accuracy,"val_f1_score": f1_score},
            on_step=False, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # Run shared step
        loss, y_hat, y, accuracy, f1_score = self._common_step(batch, batch_idx)

        # Log test metrics
        self.log_dict(
            {"test_loss": loss, "test_accuracy": accuracy,"test_f1_score": f1_score},
            on_step=False, on_epoch=True, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        """
        Prediction logic used by trainer.predict().
        Returns model outputs without computing loss.
        """
        x = batch if not isinstance(batch, (tuple, list)) else batch[0]
        logits = self.forward(x)

        # Convert logits to probabilities for inference
        probs = torch.sigmoid(logits)

        return probs

    def configure_optimizers(self):
        # Optimizer over all trainable parameters
        optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer


### **Create data module**

In [6]:
from torch.utils.data import Dataset
from PIL import Image
import torch
import pandas as pd
import os

class createMultiLabelTorchDataset(Dataset):
    def __init__(self, labels_csv_dir, img_dir, transform=None):
        self.df = pd.read_csv(labels_csv_dir)
        self.label_cols = self.df.columns.drop("Path") # Get names of labels (i.e., cols, excluding img name)
        self.img_dir     = img_dir
        self.transform   = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row["Path"]

        img_path = os.path.join(self.img_dir, image_id)
        image = Image.open(img_path).convert("RGB")  # Forces RGB (3 channels)

        if self.transform:
            image = self.transform(image)

        # multi-hot target
        target = row[self.label_cols].values.astype(float) # extract multihot target array from csv file
        target = torch.tensor(target, dtype=torch.float32) # convert multihot target array to tensor

        return image, target


class dataModule(torch_light.LightningDataModule):
  def __init__(self, batch_size, num_classes):
    super().__init__()

    self.num_classes = num_classes
    # self.validation_size = validation_size # validation_size = 0.1
    self.batch_size = batch_size

    self.train_transforms = transforms.Compose([
                transforms.RandomAffine(degrees=40, translate=(0.01, 0.12), shear=0.05),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.ToTensor(),
                # transforms.Normalize(dataConfig["MEAN"], dataConfig["STD"], inplace=True),
                transforms.RandomErasing(inplace=True)])

    self.valid_transforms = transforms.Compose([
                transforms.ToTensor(),
                # transforms.Normalize(dataConfig["MEAN"], dataConfig["STD"], inplace=True)
                ])
    self.test_transforms = self.valid_transforms

  def prepare_data(self):
      # Download datasets from Kaggle if not already downloaded
      if os.path.exists(BASE_DIR):
          return  # already downloaded

      # Kaggle expects the API key (i.e., json) to live in a directory like ~/.kaggle/kaggle.json. So we create that directory and copy the loaded Kaggle file there
      kaggle_dir  = os.path.join(os.path.expanduser("~"), ".kaggle")
      kaggle_json = os.path.join(kaggle_dir, "kaggle.json")

      if not os.path.exists(kaggle_json):
          os.makedirs(kaggle_dir, exist_ok=True)
          shutil.copyfile("kaggle.json", kaggle_json)
          os.chmod(kaggle_json, 0o600)

      print("Downloading dataset from Kaggle...")

      # Download datasets from Kaggle
      subprocess.run(
          ["kaggle", "datasets", "download","-q", "rahulogoel/nih-balanced-and-resized-chest-x-rays", "-p", BASE_DIR, "--unzip"],
          check=True)
      print("Download complete!")

  def setup(self, stage=None):

    # self.train_dataset  = createMultiLabelTorchDataset(
    #         labels_csv_dir= configs["TRAIN_CSV"],
    #         img_dir     = configs["TRAIN_DATA_DIR"],
    #         transform   = self.train_transforms,
    #         num_classes = configs["NUM_CLASSES"])

    # self.val_dataset    = createMultiLabelTorchDataset(
    #         labels_csv_dir= configs["VAL_CSV"],
    #         img_dir     = configs["VAL_DATA_DIR"],
    #         transform   = self.valid_transforms,
    #         num_classes = configs["NUM_CLASSES"])

    # self.test_dataset   = createMultiLabelTorchDataset(
    #         labels_csv_dir= configs["TEST_CSV"],
    #         img_dir     = configs["TEST_DATA_DIR"],
    #         transform   = self.test_transforms,
    #         num_classes = configs["NUM_CLASSES"])

    full_dataset = createMultiLabelTorchDataset(
        labels_csv_dir = configs["LABELS_CSV_DIR"],
        img_dir = configs["IMG_DIR"],
        transform = self.train_transforms # temporarily assign train transforms
        )

    n_total = len(full_dataset)
    train_set_size = int(configs["TRAIN_PCT"] * n_total)
    val_set_size   = int(configs["VAL_PCT"] * n_total)
    test_set_size  = n_total - train_set_size - val_set_size

    generator = torch.Generator().manual_seed(42)

    train_subset, val_subset, test_subset = random_split(
        full_dataset,[train_set_size, val_set_size, test_set_size],
        generator=generator)

    # Assign transforms AFTER split
    train_subset.dataset.transform = self.train_transforms
    val_subset.dataset.transform   = self.valid_transforms
    test_subset.dataset.transform  = self.test_transforms

    if stage in (None, "fit"):
        self.train_dataset = train_subset
        self.val_dataset   = val_subset

    if stage in (None, "test"):
        self.test_dataset  = test_subset


  def train_dataloader(self):
    # create training data loader object from training dataset
    return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)


In [11]:
model_ = modelModule(
    num_classes = configs["NUM_CLASSES"],
    backbone_model_name = "efficientnet_v2_s")
    # efficientnet_v2_s, convnext_small, alexnet, resnet50, vgg11

data_module = dataModule(
    batch_size      = configs["BATCH_SIZE"],
    num_classes     = configs["NUM_CLASSES"])

trainer_ = torch_light.Trainer(
    accelerator = "auto",
    devices     = "auto",
    strategy    = "auto",
    max_epochs  = 50,
    callbacks   = [EarlyStopping(patience=5, verbose=False, monitor="val_loss")],
    precision   = "16")

trainer_.fit(model_, data_module)
trainer_.validate(model_, data_module)
trainer_.test(model_, data_module)


Loading pretrained [efficientnet_v2_s] model


/usr/local/lib/python3.12/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.

┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name              ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model             │ EfficientNet       │ 20.2 M │ train │     0 │
│ 1 │ loss_function     │ BCEWithLogitsLoss  │      0 │ train │     0 │
│ 2 │ accuracy_function │ MultilabelAccuracy │      0 │ train │     0 │
│ 3 │ f1_score_function │ MultilabelF1Score  │      0 │ train │     0 │
└───┴───────────────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 19.2 K                                                                                           
Non-trainable params: 20.2 M                                                                                       
Total params: 20.2 M                                                                                               
Total estimated model params size (MB): 80                                                                         
Modules in train mode: 717                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

...

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9047936201095581     │
│       test_f1_score       │    0.12528811395168304    │
│         test_loss         │    0.28348255157470703    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.28348255157470703,
  'test_accuracy': 0.9047936201095581,
  'test_f1_score': 0.12528811395168304}]